<a href="https://colab.research.google.com/github/MeidanGR/SER_DeepLearning_LSTM/blob/main/EmotionRecognition_DL_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Speech Emotion Recognition algorithm**
*Using Deep Learning (LSTM) model*

B.Sc. Final project by Meidan Greenberg; Linoy Hadad;

Instructor: Dr. Dima Alberg

# UNDER DEVELOPMENT.

# **PACKAGES & GOOGLE AUTH**

In [3]:
%%capture
!pip install soundfile
!pip install noisereduce

In [4]:
import numpy as np
import librosa
import soundfile as sf
import noisereduce as nr
import IPython.display as ipd #Audio player
import os
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# **LOADING AUDIO FILES**
## **TRAIN SET** = RAVDESS Database
All of  RAVDESS files has a unique filename. The filename consists of a 7-part numerical identifier (e.g., 02-01-06-01-02-01-12.mp4). These identifiers define the stimulus characteristics: 

### Filename identifiers 

Modality (01 = full-AV, 02 = video-only, **03 = audio-only**).

Vocal channel (**01 = speech**, 02 = song).

**Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).**

Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.

Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").

Repetition (01 = 1st repetition, 02 = 2nd repetition).

Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [6]:
#Importing only files with format: 03-01-XX-XX-XX-XX-XX.WAV

folder_path = '/content/drive/My Drive/AudioFiles/RAVDESS'
audio_data = []
emotions = []
sample_rate = []

for subdir, dirs, files in os.walk(folder_path):
  for file in files:
    try:
      x, sr = sf.read(os.path.join(subdir,file))  #Loading the audio frames into x, samplerate into sr.
      #f1
      #f2
      #f3
      #f4
      #f5
      name = file[6:8]  #Fetching emotion num from the file name.

      audio_data.append(x.T) #Adding each audio file signals into a list.
      #feature1.append(f1)
      #f2
      #f3
      #f4
      #f5
      emotions.append(name)  #Adding each emotion into a list.
      sample_rate.append(sr) #Adding each sample rate value to a list.

    except ValueError:
      continue

train_set_sr = np.mean(sample_rate) #Mean value of all sample rates in the data, if all the data is with the same sr, it should be that value, for e.g 48kHz.


df = pd.DataFrame(columns= ['x_train', 'y_train']) #DataFrame is used to 1) Visualization, 
#                                                                        2) Converting emotion number into a name using 'map' function.

df.x_train = audio_data
df.y_train = emotions
df.y_train = df.y_train.map({'01' : 'neutral', '02' : 'calm', '03' : 'happy', '04' : 'sad', '05' : 'angry',
                             '06' : 'fearful', '07' : 'disgust', '08' : 'suprised'})



x_train = df.x_train #Converting to a list for convenience of use.
y_train = df.y_train #Converting to a list for convenience of use.

df.sample(10)

,x_train,y_train
863,"[-6.103515625e-05, -6.103515625e-05, -9.155273...",angry
1149,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",happy
956,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",suprised
206,"[-9.1552734375e-05, -9.1552734375e-05, -3.0517...",sad
766,"[0.0, 0.0, 0.0, 0.0, 0.0, -3.0517578125e-05, 0...",fearful
1432,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",disgust
548,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",happy
1192,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",suprised
144,"[0.0, -6.103515625e-05, -6.103515625e-05, -3.0...",happy
823,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",fearful


In [16]:
row = 1439

print('Emotion:', y_train[row])
ipd.display(ipd.Audio(data = x_train[row], rate=train_set_sr))

print(train_set_sr, 'Hz')
print(x_train.shape)

Emotion: suprised


48000.0 Hz
(1440,)


## **TEST SET** = TESS Database
Emotions in this database named (relevant names & number from RAVDESS): 

anger (anger, 05), disgust (disgust, 07), fear (fearful, 06), happiness (happy, 03), pleasant surprise (suprised, 08) , sadness (sad, 04), and neutral (neutral, 01)

In [41]:
def find_emotion(name): 
        if('neutral' in name): return "01"
        elif('happy' in name): return "03"
        elif('sad' in name): return "04"
        elif('angry' in name): return "05"
        elif('fear' in name): return "06"
        elif('disgust' in name): return "07"
        elif('ps' in name): return "08"




folder_path = '/content/drive/My Drive/AudioFiles/TESS'
audio_data = []
emotions = []
sample_rate = []


for subdir, dirs, files in os.walk(folder_path):
  for file in files:
    try:
      x, sr = sf.read(os.path.join(subdir,file))  #Loading the audio frames into x, samplerate into sr.
      name = find_emotion(str(file)) #Extracting emotion from file name

      audio_data.append(x.T) #Adding each audio file signals into a list.
      emotions.append(name)  #Adding each emotion into a list.
      sample_rate.append(sr) #Adding each sample rate value to a list.

    except ValueError:
      continue

df = pd.DataFrame(columns= ['x_test', 'y_test']) #DataFrame is used to 1) Visualization, 2) Converting emotion number into a name. 

df.x_test = audio_data
df.y_test = emotions
df.y_test = df.y_test.map({'01' : 'neutral', '03' : 'happy', '04' : 'sad', '05' : 'angry',
                             '06' : 'fearful', '07' : 'disgust', '08' : 'suprised'}) #Datebase do not posses 'calm' emotion

test_set_sr = np.mean(sample_rate) #Mean value of all sample rates in the data, if all the data is with the same sr, it should be that value, for e.g 48kHz.

x_test = df.x_test #Converting to a list
y_test = df.y_test #Converting to a list

df.sample(10)


,x_test,y_test
2025,"[0.0, -9.1552734375e-05, -0.0001220703125, -6....",neutral
569,"[9.1552734375e-05, -6.103515625e-05, 9.1552734...",fearful
874,"[0.000152587890625, -0.002288818359375, -0.002...",happy
2148,"[0.0, -6.103515625e-05, -6.103515625e-05, -9.1...",fearful
2465,"[0.0, -0.001129150390625, -0.00225830078125, -...",sad
1537,"[0.0, -0.0001220703125, -0.0001220703125, -0.0...",neutral
593,"[6.103515625e-05, -0.000274658203125, -0.00021...",fearful
1171,"[0.0, -0.0001220703125, -0.000152587890625, -0...",angry
1510,"[0.0, -6.103515625e-05, -9.1552734375e-05, -9....",sad
849,"[-3.0517578125e-05, -0.0001220703125, 3.051757...",disgust


In [18]:
row = 2

print('Emotion:', y_test[row])
ipd.display(ipd.Audio(data = x_test[row], rate=test_set_sr))

print(test_set_sr)
print(x_test.shape)


NameError: ignored